In [1]:
titanic_url = "https://raw.githubusercontent.com/dhavalsays/py/refs/heads/master/ML/14_naive_bayes/titanic.csv"

In [13]:
import pandas as pd

titanic_df = pd.read_csv(titanic_url)
titanic_df.head()

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


In [14]:
titanic_df.drop(
    ["PassengerId", "Name", "SibSp", "Parch", "Ticket", "Cabin", "Embarked"],
    axis="columns",
    inplace=True,
)

titanic_df.rename(
    columns={
        "Survived": "survived"
        , "Pclass": "class"
        , "Sex": "sex"
        , "Age": "age"
        , "Fare": "fare"
        },
    inplace=True,
)

titanic_df.head()

,class,sex,age,fare,survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


In [15]:
titanic_df['sex'] = titanic_df['sex'].map({'male': 0, 'female': 1})
titanic_df.head()

,class,sex,age,fare,survived
0,3,0,22.0,7.2500,0
1,1,1,38.0,71.2833,1
2,3,1,26.0,7.9250,1
3,1,1,35.0,53.1000,1
4,3,0,35.0,8.0500,0


In [16]:
import math

mean_age = math.floor(titanic_df.age.mean())
titanic_df.age = titanic_df.age.fillna(mean_age)
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   class     891 non-null    int64  
 1   sex       891 non-null    int64  
 2   age       891 non-null    float64
 3   fare      891 non-null    float64
 4   survived  891 non-null    int64  
dtypes: float64(2), int64(3)
memory usage: 34.9 KB


In [17]:
ip = titanic_df.drop('survived', axis='columns')
op = titanic_df.survived

In [21]:
from sklearn.model_selection import train_test_split

ip_train, ip_test, op_train, op_test = train_test_split(ip, op, test_size=0.2)

In [22]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(ip_train, op_train)

GaussianNB()

In [23]:
model.score(ip_test, op_test)

0.7653631284916201

# Email Spam Detection

In [24]:
spam_url = "https://raw.githubusercontent.com/dhavalsays/py/refs/heads/master/ML/14_naive_bayes/spam.csv"

In [28]:
spam_df = pd.read_csv(spam_url)
spam_df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [29]:
spam_df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [30]:
spam_df['is_spam'] = spam_df['Category'].apply(lambda x: 1 if x == 'spam' else 0)

spam_df.drop('Category', axis='columns', inplace=True)
spam_df.rename(
    columns={
        'Message': 'message'
        }
    , inplace=True)

spam_df.head()

,message,is_spam
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [31]:
from sklearn.model_selection import train_test_split

ip_train, ip_test, op_train, op_test = train_test_split(spam_df.message, spam_df.is_spam)

In [32]:
# we shall convert the message text field into a number field, using countVectorizer

from sklearn.feature_extraction.text import CountVectorizer

v = CountVectorizer()
ip_train_count = v.fit_transform(ip_train.values)

In [34]:
ip_train_count.toarray()[0]

array([0, 0, 0, ..., 0, 0, 0])

This has taken into account, all the distinct words in our complete text, and then made a column for each of them.

In [35]:
ip_train_count.shape

(4179, 7542)

So we have a total of 4179 rows (in training) and they contain 7542 distinct words

In [36]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(ip_train_count, op_train)

MultinomialNB()

In [37]:
test_emails = [
    'Hey mohan, can we get together to watch footbal game tomorrow?',
    'Upto 20% discount on parking, exclusive offer just for you. Dont miss this reward!'
]
test_emails_count = v.transform(test_emails)
model.predict(test_emails_count)

array([0, 1])

In [38]:
ip_test_count = v.transform(ip_test)
model.score(ip_test_count, op_test)

0.9849246231155779

In [39]:
# sometimes we have multiple transformations to perform before predicting, those times we use a pipeline

from sklearn.pipeline import Pipeline

clf = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('nb', MultinomialNB())
])

In [40]:
clf.fit(ip_train, op_train)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [41]:
clf.score(ip_test, op_test)

0.9849246231155779

In [42]:
clf.predict(test_emails)

array([0, 1])

# Wine Classification

In [43]:
from sklearn.datasets import load_wine

wine = load_wine()
wine

{'data': array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
         1.065e+03],
        [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
         1.050e+03],
        [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
         1.185e+03],
        ...,
        [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
         8.350e+02],
        [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
         8.400e+02],
        [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
         5.600e+02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [44]:
wine_df = pd.DataFrame(wine.data, columns=wine.feature_names)
wine_df.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0


In [45]:
ip = wine.data
op = wine.target

In [55]:
from sklearn.model_selection import train_test_split

ip_train, ip_test, op_train, op_test = train_test_split(ip, op, test_size=0.2)

## Gaussian BM

In [56]:
from sklearn.naive_bayes import GaussianNB

gnb_model = GaussianNB()
gnb_model.fit(ip_train, op_train)

GaussianNB()

In [57]:
gnb_model.score(ip_test, op_test)

1.0

## Multinomial Classifier BM

In [58]:
from sklearn.naive_bayes import MultinomialNB

mnb_model = MultinomialNB()
mnb_model.fit(ip_train, op_train)

MultinomialNB()

In [59]:
mnb_model.score(ip_test, op_test)

0.8611111111111112

| Feature Type          | GaussianNB                      | MultinomialNB                  |
|-----------------------|----------------------------------|---------------------------------|
| **Data Type**         | Continuous                      | Discrete (counts, frequencies) |
| **Assumption**        | Gaussian (normal) distribution  | Multinomial distribution       |
| **Use Case Examples** | Medical data, stock prediction  | Text data, document classification |
| **Typical Feature Values** | Real numbers               | Non-negative integers          |
